In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import sys
from tqdm import tqdm
import h5py

In [2]:
training_set = pd.read_csv("Data/training_set.csv")
training_set_metadata = pd.read_csv("Data/training_set_metadata.csv")

print(training_set.shape, training_set_metadata.shape)

(1421705, 6) (7848, 12)


In [23]:
unique_obj_ids = np.unique(training_set["object_id"]).tolist()

hf = h5py.File("Data/Data_as_h5/obj_passbands.h5", "w")
for j in tqdm( range( len(unique_obj_ids) ) ):
    obj_id = unique_obj_ids[j]
    training_set_obj_id = training_set.loc[training_set["object_id"] == obj_id,:]
    passbands_counts = training_set_obj_id["passband"].value_counts()
    
    # Get unique passbands for that object
    unique_passbands = np.unique(training_set_obj_id["passband"]).tolist()
    for i in range( len(unique_passbands) ):
        # Focus on one
        passband = unique_passbands[i]
        # Filter the dataframe of the object for that passband
        training_set_obj_id_passband = training_set_obj_id.loc[training_set_obj_id["passband"] == passband,:]
        training_set_obj_id_passband = training_set_obj_id_passband.drop(["object_id", "passband", "detected"], axis = 1)
        hf.create_dataset("objid_" + str(obj_id) + "_passband_" + str(passband), data = np.array(training_set_obj_id_passband.values))
hf.close()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7848/7848 [01:06<00:00, 117.91it/s]


In [28]:
def load_passbands_list(path, objid):
    passbands_list = []
    hf = h5py.File(path, "r")
    for i in range(6):
        passbands_list.append( np.array( hf.get("objid_" + str(objid) + "_passband_" + str(i)) ) )
    hf.close()
    return passbands_list

In [33]:
xd = load_passbands_list(path = "Data/Data_as_h5/obj_passbands.h5", objid = 615)

xd[1].shape

(58, 3)

In [24]:
hf = h5py.File("Data/Data_as_h5/obj_passbands.h5", "r")
print(hf.keys())
hf.close()

<KeysViewHDF5 ['objid_100012747_passband_0', 'objid_100012747_passband_1', 'objid_100012747_passband_2', 'objid_100012747_passband_3', 'objid_100012747_passband_4', 'objid_100012747_passband_5', 'objid_100017743_passband_0', 'objid_100017743_passband_1', 'objid_100017743_passband_2', 'objid_100017743_passband_3', 'objid_100017743_passband_4', 'objid_100017743_passband_5', 'objid_100018744_passband_0', 'objid_100018744_passband_1', 'objid_100018744_passband_2', 'objid_100018744_passband_3', 'objid_100018744_passband_4', 'objid_100018744_passband_5', 'objid_100045757_passband_0', 'objid_100045757_passband_1', 'objid_100045757_passband_2', 'objid_100045757_passband_3', 'objid_100045757_passband_4', 'objid_100045757_passband_5', 'objid_100057_passband_0', 'objid_100057_passband_1', 'objid_100057_passband_2', 'objid_100057_passband_3', 'objid_100057_passband_4', 'objid_100057_passband_5', 'objid_100097_passband_0', 'objid_100097_passband_1', 'objid_100097_passband_2', 'objid_100097_passband